In [43]:
from scipy.optimize import linear_sum_assignment
import random
from collections import defaultdict
import numpy as np
import time
import pickle
import csv
import scipy as sp
import pandas as pd

In [87]:
def parse_yelp(num_users_str):
    file_path = './yelp_dataset_' + num_users_str + '_users.pkl'
    infile = open(file_path,'rb')
    df = pickle.load(infile)
    G = defaultdict(dict)
    infile.close()

    user_id_map, review_id_map = {}, {}
    unique_user_ids, unique_review_ids = df.user_id.unique(), df.review_id.unique()
    for i, user_id in enumerate(unique_user_ids):
        user_id_map[user_id] = i
    for i, review_id in enumerate(unique_review_ids):
        review_id_map[review_id] = i  

    for user_id, review_id in zip(df['user_id'], df['review_id']):
        G[user_id][review_id] = 1
    
    matrix = [[0 for j in range(len(unique_review_ids))] for i in range(len(unique_user_ids))]
    
    for user_id in G:
        for review_id in G[user_id].keys():
            matrix[user_id_map[user_id]][review_id_map[review_id]] = 1

    return matrix
    

matrix_8000 = parse_yelp('8000')


In [88]:
def compute_maximum_matching(A):
    start = time.time()
    A = np.array(A)
    row_ind, col_ind = linear_sum_assignment(A, maximize=True)
    end = time.time()
    print("Run Time: ", end - start, "seconds")
    return A[row_ind, col_ind].sum()

print(compute_maximum_matching(matrix_8000))

Run Time:  4.715924978256226 seconds
7548


In [22]:
def create_bipartite_graph(x, y, num_edges):
    matrix = [[0 for j in range(y)] for i in range(x)]
    for i in range(x):
        indices = list(range(y))
        choices = set(random.sample(indices, k=num_edges))
        for j in range(y):
            if j in choices:
                matrix[i][j] = 1
    return matrix


In [23]:
x, y, num_edges = 1000, 10000, 2
G = create_bipartite_graph(x, y, num_edges)
for row in G:
    print(row)
print("Computed", x, y, num_edges)
print("Maximum Matching: ", compute_maximum_matching(G))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Run Time:  0.7442131042480469 seconds
Maximum Matching:  1000


In [24]:
x, y, num_edges = 100, 100, 1
G = create_bipartite_graph(x, y, num_edges)
print("Computed", x, y, num_edges)
print("Maximum Matching: ", compute_maximum_matching(G))

Computed 100 100 1
Run Time:  0.0017778873443603516 seconds
Maximum Matching:  60


In [25]:
x, y, num_edges = 1000, 1000, 10
G = create_bipartite_graph(x, y, num_edges)
print("Computed", x, y, num_edges)
print("Maximum Matching: ", compute_maximum_matching(G))

Computed 1000 1000 10
Run Time:  0.08113217353820801 seconds
Maximum Matching:  1000


In [29]:
def parse_sports():
    rows = []
    file = open('players.csv')
    csvreader = csv.reader(file)
    for row in csvreader:
        rows.append(row)
    rows = rows[1:]
    country_idx_map, i = {}, 0
    for source, dest, _ in rows:
        if source not in country_idx_map:
            country_idx_map[source] = i
            i += 1
        if dest not in country_idx_map:
            country_idx_map[dest] = i
            i += 1
    num_countries = len(country_idx_map)
    matrix = [[0 for i in range(num_countries)]\
                 for j in range(num_countries)]
    
    for source, dest, _ in rows:
        matrix[country_idx_map[source]][country_idx_map[dest]] = 1
    
    return matrix
        

sports_matrix = parse_sports()
print(compute_maximum_matching(sports_matrix))

Run Time:  0.0004937648773193359 seconds
32


In [33]:
def parse_members():
    rows = []
    file = open('member-to-group-edges.csv')
    csvreader = csv.reader(file)
    for row in csvreader:
        rows.append(row)
    rows = rows[1:]
    idx_map, i = {}, 0
    for source, dest, _ in rows:
        if source not in idx_map:
            idx_map[source] = i
            i += 1
        if dest not in idx_map:
            idx_map[dest] = i
            i += 1
    n = len(idx_map)
    matrix = [[0 for i in range(n)]\
                 for j in range(n)]
    
    for source, dest, _ in rows:
        matrix[idx_map[source]][idx_map[dest]] = 1
    
    return matrix

members_matrix = parse_members()
print(compute_maximum_matching(members_matrix))

Run Time:  68.944495677948 seconds
601


In [86]:
def parse_gn():
    rows, idx_map, i = [], {}, 0
    data = pd.read_csv ("out.edit-gnwikibooks", sep = '\t')
    for row in data.iterrows():
        rows.append([row[0][0], row[0][1]])
        
    idx_map, i = {}, 0
    for source, dest in rows:
        if source not in idx_map:
            idx_map[source] = i
            i += 1
        if dest not in idx_map:
            idx_map[dest] = i
            i += 1
    n = len(idx_map)
    matrix = [[0 for i in range(n)]\
                 for j in range(n)]
    
    for source, dest in rows:
        matrix[idx_map[source]][idx_map[dest]] = 1
    
    return matrix
gn_matrix = parse_gn()
print(compute_maximum_matching(gn_matrix))

Run Time:  0.00026106834411621094 seconds
20


In [103]:
def parse_bronson():
    rows, idx_map, i = [], {}, 0
    data = pd.read_csv ("out.brunson_club-membership_club-membership.tsv", sep = '\t')
    for row in data.iterrows():
        split = row[1][0].split(" ")
        rows.append([split[0], split[1]])
    idx_map, i = {}, 0
    for source, dest in rows:
        if source not in idx_map:
            idx_map[source] = i
            i += 1
        if dest not in idx_map:
            idx_map[dest] = i
            i += 1
    n = len(idx_map)
    matrix = [[0 for i in range(n)]\
                 for j in range(n)]
    
    for source, dest in rows:
        matrix[idx_map[source]][idx_map[dest]] = 1
    
    return matrix
bronson_matrix = parse_bronson()
print(compute_maximum_matching(bronson_matrix))

Run Time:  0.00012373924255371094 seconds
15
